In [ ]:
# Montagem do cadastro de empresas com todos dados necessários de cruzamento
    # Codigo da CVM que há nos balanços
    # Ticker da B3 para consultar cotações e ligar valores de ON e PN
    # CNPJ que nesse caso serve para fazer a ponte entre os 2 acima

import pandas as pd

In [ ]:
# Obter Dados

emp_cvm_cnpj = pd.read_excel('AUXILIAR\Empresas_CD_CVM_e_CNPJ.xlsx',converters={'CD_CVM':str})

emp_ticker_cnpj = pd.read_excel('AUXILIAR\Empresas_Ticker_CNPJ.xlsx')
emp_ticker_cnpj["CNPJ"] = emp_ticker_cnpj["CNPJ"].str.slice(1, 19, 1) # pequeno ajuste no formato do CNPJ

emp_setores = pd.read_excel('AUXILIAR\Empresas_Setores.xlsx', skiprows=5, header=1)


In [ ]:
# Primeiro merge para conectar codigo da CVM com codigo de ticker

df_emp = emp_cvm_cnpj.merge(
            emp_ticker_cnpj,
            left_on='CNPJ_CIA', 
            right_on='CNPJ', 
            how='left').drop(
                ['Denominação social','Nome de pregão','CNPJ'],
                axis=1)

In [ ]:
# Limpeza da tabela de setores das empresas

emp_setores['SETOR ECONÔMICO'] = emp_setores['SETOR ECONÔMICO'].ffill()
emp_setores['SUBSETOR'] = emp_setores['SUBSETOR'].ffill()
emp_setores = emp_setores[emp_setores['LISTAGEM'].notna()]
emp_setores = emp_setores[(emp_setores['LISTAGEM'] != 'CÓDIGO') & (emp_setores['LISTAGEM'] != 'LISTAGEM')]
emp_setores.drop(emp_setores.columns[-1],axis=1, inplace=True)

In [ ]:
# Segundo merge para obter segmentos

df_emp = df_emp.merge(emp_setores, left_on='Sigla', right_on='LISTAGEM', how='left')

In [ ]:
# Obter tickers de ON e PN disponóveis

df_invespty = pd.read_excel('AUXILIAR\Empresas_investpy.xlsx')
df_invespty = df_invespty[df_invespty['symbol'].str.len() == 5] # deixar somente tickers com 5 caracteres, 4 letras e 1 numero
df_invespty_PN = df_invespty[df_invespty['symbol_type'] == 'PN'][['symbol','symbol_main']]
df_invespty_PN.drop_duplicates('symbol', inplace=True)  #caso tenha algum ticker duplicado, eliminar
df_invespty_ON = df_invespty[df_invespty['symbol_type'] == 'ON'][['symbol','symbol_main']]
df_invespty_ON.drop_duplicates('symbol', inplace=True) #caso tenha algum ticker duplicado, eliminar

In [71]:
df_emp = df_emp.merge(df_invespty_ON, left_on='LISTAGEM',right_on='symbol_main', how='left' # merge com os ticker PN
              ).merge(df_invespty_PN, left_on='LISTAGEM',right_on='symbol_main', how='left' # merge com os ticker ON
              ).drop(['symbol_main_x','symbol_main_y'], axis=1).rename( # dropar colunas repetidas
                columns={'symbol_x':'ON','symbol_y':'PN'} # Renomear colunas para organizar colunas de ON e PN
              )

In [72]:
# Salvar em excel

# df_emp.to_excel('AUXILIAR/Empresas_Cadastro_Full.xlsx')

In [73]:
df_emp

,CD_CVM,DENOM_CIA,CNPJ_CIA,Sigla,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,ON,PN,ON,PN,ON,PN
0,2437,CENTRAIS ELET BRAS S.A. - ELETROBRAS,00.001.180/0001-26,ELET,Utilidade Pública,Energia Elétrica,ELETROBRAS,ELET,ELET3,ELET6,ELET3,ELET6,ELET3,ELET6
1,14451,CIA ENERGETICA DE BRASILIA,00.070.698/0001-11,CEBR,Utilidade Pública,Energia Elétrica,CEB,CEBR,CEBR3,NaN,CEBR3,NaN,CEBR3,NaN
2,14540,VBC ENERGIA SA,00.095.147/0001-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14621,NET SERVIÇOS DE COMUNICAÇÃO S.A.,00.108.786/0001-65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14796,FORPART S.A. - EM LIQUIDAÇÃO,00.249.786/0001-85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15083,KOSMOS COMÉRCIO DE VESTUÁRIO S/A - EM RECUPERA...,00.354.053/0001-00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,22594,STATKRAFT ENERGIAS RENOVAVEIS S.A.,00.622.416/0001-41,STKF,Utilidade Pública,Energia Elétrica,STATKRAFT,STKF,NaN,NaN,NaN,NaN,NaN,NaN
7,15091,LITEL PARTICIPACOES S.A.,00.743.065/0001-27,LTEL,Materiais Básicos,Mineração,LITEL,LTEL,NaN,NaN,NaN,NaN,NaN,NaN
8,20990,AMERICANAS S.A.,00.776.574/0001-56,BTOW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,15253,ENERGISA S.A.,00.864.214/0001-06,ENGI,Utilidade Pública,Energia Elétrica,ENERGISA,ENGI,ENGI3,ENGI4,ENGI3,ENGI4,ENGI3,ENGI4
